In [1]:
import os
import torch
import numpy as np
import time
import random
from sklearn.model_selection import KFold
import argparse
import timeit
from dataset import prepare_data
import psutil
import torch
from models.gconvGRU_simple import GConvGRUModel
from memory_capacity_utils import gen_lag_data, get_mem_cap_from_model

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('running on GPU')
else:
    device = torch.device("cpu")
    print('running on CPU')


running on GPU


# Load the dataset

In [4]:
dataset = np.load('datasets/oasis_data.npy')
dataset = torch.from_numpy(dataset).squeeze()
dataset = dataset.type(torch.FloatTensor)
dataset = np.delete(dataset,88,axis=0)

# dataset = np.load("multivariate_simulation_data_3.npy")
# dataset = torch.from_numpy(dataset).squeeze()
# dataset = dataset.type(torch.FloatTensor)

In [5]:
def get_args():
    parser = argparse.ArgumentParser(description='Args for graph predition')
    parser.add_argument('-num_folds', type=int, default=3, help='cv number')
    parser.add_argument('--num_timepoints', type=int, default=3,
                        help='Number of timepoints')
    parser.add_argument('-num_epochs', type=int, default=20, help='number of epochs')
    parser.add_argument('--lr', type=float, default=0.001, help="Learninng rate")
    parser.add_argument('--memcap_coef', type=float, default=0, help="Memory Capacity Loss Coefficient")
    parser.add_argument('-max_lag', type=int, default=35, help='Lag tao for memory capacity signals')
    parser.add_argument('-save_path',type=str,default = '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_t/',help='Path to the saved results')
    args, _ = parser.parse_known_args()
    return args

In [6]:
args = get_args()

# Set up the network architecture and train-validate

**Train-validate-functions**

In [7]:
manual_seed = 777
np.random.seed(manual_seed)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

# Reservoir signals
X_train_res_np, y_train_res_np = gen_lag_data(1000, 41, args.max_lag)
X_test_res_np, y_test_res_np = gen_lag_data(500, 42, args.max_lag)
X_train_res = torch.from_numpy(X_train_res_np).unsqueeze(1).to(device, dtype=torch.float64)
X_test_res = torch.from_numpy(X_test_res_np).unsqueeze(1).to(device, dtype=torch.float64)
y_train_res = torch.from_numpy(y_train_res_np).to(device, dtype=torch.float64)
y_test_res = torch.from_numpy(y_test_res_np).to(device, dtype=torch.float64)

In [8]:
def create_directory_if_not_exists(directory):
    """
    Checks if a specified directory exists, and creates it if it doesn't.

    Args:
    - directory (str): Path of the directory to check and potentially create.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' was created.")
    else:
        print(f"Directory '{directory}' already exists.")

# Create the results folders
create_directory_if_not_exists(args.save_path+'real_and_predicted_graphs')
create_directory_if_not_exists(args.save_path+'train_losses/mae_losses')
create_directory_if_not_exists(args.save_path+'train_losses/reservoir_losses')
create_directory_if_not_exists(args.save_path+'train_losses/bio_losses')
create_directory_if_not_exists(args.save_path+'train_losses/tp_losses')
create_directory_if_not_exists(args.save_path+'train_losses/total_losses')
create_directory_if_not_exists(args.save_path+'test_mae_losses')
create_directory_if_not_exists(args.save_path+'test_tp_losses')
create_directory_if_not_exists(args.save_path+'test_memcap_losses')
create_directory_if_not_exists(args.save_path+'test_predicted')
create_directory_if_not_exists(args.save_path+'test_original')
create_directory_if_not_exists(args.save_path+'trained_models')

Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_t/real_and_predicted_graphs' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_t/train_losses/mae_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_t/train_losses/reservoir_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_t/train_losses/bio_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_t/train_losses/tp_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_t/train_losses/total_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_t/test_mae_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_t/test_tp_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_t/test_

In [29]:
def validation(model, validation_subjects, mem_cap_data, X_train, y_train, X_test, y_test):
    mael = torch.nn.L1Loss().to(device)
    tp = torch.nn.MSELoss().to(device)
    
    val_mae_loss = np.zeros(args.num_timepoints - 1)
    val_tp_loss = np.zeros(args.num_timepoints - 1)
    mem_cap = np.zeros(args.num_timepoints - 1)
    predicted = np.zeros((validation_subjects.shape[0], args.num_timepoints - 1, 35, 35))
    actual = np.zeros((validation_subjects.shape[0], args.num_timepoints - 1, 35, 35))
    
    model.eval()

    with torch.no_grad():
        for n_subject, data in enumerate(validation_subjects):
            input = data[0]
            for t in range(args.num_timepoints - 1):
                pred = model(input)
                val_mae_loss[t] += mael(pred, data[t + 1])
                val_tp_loss[t] += tp(pred.sum(dim=-1), data[t + 1].sum(dim=-1))
                input = pred
                
                pred_mem_cap = get_mem_cap_from_model(model, pred, 
                                                      X_train, y_train, X_test, y_test)
                actual_mem_cap = torch.tensor(mem_cap_data[n_subject, t + 1]).to(device)
                mem_cap[t] += torch.abs(pred_mem_cap - actual_mem_cap)

                predicted[n_subject, t] = pred.cpu().detach().numpy()
                actual[n_subject, t] = data[t + 1].cpu().detach().numpy()
                

    avg_val_mae_loss = val_mae_loss/len(validation_subjects)
    avg_val_tp_loss = val_tp_loss/len(validation_subjects)
    avg_val_mae_mem_cap = mem_cap/len(validation_subjects)

    return avg_val_mae_loss, avg_val_tp_loss, avg_val_mae_mem_cap, predicted, actual

In [31]:
torch.manual_seed(1)
input_weights = (torch.rand((35, 1), dtype=torch.float64) * 2.0 - 1.0).to(device)

indexes = range(args.num_folds)
kfold = KFold(n_splits=args.num_folds, shuffle=True, random_state=manual_seed)
dataset = dataset.to(device)
actual_mem_caps = np.load('datasets/mem_caps_oasis.npy')
f = 0

for train, test in kfold.split(range(dataset.shape[0])):
    print(
            f'------------------------------------Fold [{f + 1}/{args.num_folds}]-----------------------------------------')
    
    train_data = dataset[train]
    test_data = dataset[test]
    train_mem_cap = actual_mem_caps[train]
    test_mem_cap = actual_mem_caps[test]
    
    validation_split = int(0.8 * len(train_data))
    train_subjects = train_data[:validation_split]
    train_mem_cap_subjects = train_mem_cap[:validation_split]
    validation_subjects = train_data[validation_split:]
    validation_mem_cap_subjects = train_mem_cap[:validation_split]

    # Create model instance
    model = GConvGRUModel(device=device, input_weights=input_weights).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    mael = torch.nn.L1Loss().to(device)
    tp = torch.nn.MSELoss().to(device)
  
    # Start measuring the epochs time
    epochs_start = time.time()
    for epoch in range(args.num_epochs):
        
        print(f'Epoch [{epoch + 1}/{args.num_epochs}]')
        # Set the model in training mode
        model.train()
        
        # this is our loss for all the data
        mae_loss_overall = []
        tp_loss_overall = []
        
        # loop through the data batches
        for data_id, data in enumerate(train_subjects):

            # zero the gradients
            optimizer.zero_grad()
            mae_loss = 0
            tp_loss = 0
         
            # loop through the time dependent adj matrices in the batches
            for t in range(args.num_timepoints - 1): 
                pred = model(data[t])
                real = data[t + 1]
                # print(pred)
                mae_loss += mael(pred, real)

                # Topological Loss
                tp_loss += tp(pred.sum(dim=-1), real.sum(dim=-1))

    
            # Calculate the total MAE Loss for the current batch
            mae_loss = mae_loss / (args.num_timepoints - 1)

            # Calculate the total TP Loss for the current batch
            tp_loss = tp_loss / (args.num_timepoints - 1)

            # Append to the total MAE Loss
            mae_loss_overall.append(mae_loss.item())
            tp_loss_overall.append(tp_loss.item())
            
            total_loss = mae_loss
            # Update the weights of the neural network
            total_loss.backward()
            optimizer.step()

        mae_loss_overall = np.mean(np.array(mae_loss_overall))
        tp_loss_overall = np.mean(np.array(tp_loss_overall))
        print(f"[Train] MAE Loss: {mae_loss_overall}, TP Loss: {tp_loss_overall}")
    
        avg_val_mae_loss, avg_val_tp_loss, avg_val_mae_mem_cap, _, _ = validation(model, validation_subjects, validation_mem_cap_subjects,
                                                                                  X_train_res, y_train_res, X_test_res, y_test_res)
        print(f"[Validate] MAE Loss Across Timepoints: {avg_val_mae_loss}")
        print(f"[Validate] TP Loss Across Timepoints: {avg_val_tp_loss}")
        print(f"[Validate] MAE of Mem Caps Across Timepoints: {avg_val_mae_mem_cap}")

    
    epochs_end = time.time() - epochs_start
    print()
    print(f'epochs finished with time:{epochs_end}')
    print()
    process = psutil.Process(os.getpid())
    print(f"Current memory usage: {process.memory_info().rss / 1024 ** 2:.2f} MB")
    print()
    
    avg_test_mae_loss, avg_test_tp_loss, avg_test_mem_cap, predicted, original = validation(model, test_data, test_mem_cap, 
                                                                                            X_train_res, y_train_res, X_test_res, y_test_res)
    print(f"[Test] MAE Loss Across Timepoints: {avg_test_mae_loss}")
    print(f"[Test] TP Loss Across Timepoints: {avg_test_tp_loss}")
    print(f"[Test] MAE of Mem Caps Across Timepoints: {avg_test_mem_cap}")
    np.save(args.save_path+f"test_mae_losses/mae_test_loss_fold_{f}", avg_test_mae_loss)
    np.save(args.save_path+f"test_tp_losses/tp_test_loss_fold_{f}", avg_test_mae_loss)
    np.save(args.save_path+f"test_memcap_losses/memcap_test_loss_fold_{f}", avg_test_mem_cap)
    np.save(args.save_path+f"test_predicted/predicted_fold_{f}", predicted)
    np.save(args.save_path+f"test_original/original_fold_{f}", original)

    torch.save(model.state_dict(),
               args.save_path +f'trained_models/model_fold_{f}')
    f += 1
    

------------------------------------Fold [1/3]-----------------------------------------
Epoch [1/20]
[Train] MAE Loss: 0.04047726597636938, TP Loss: 2.7933480749527613
[Validate] MAE Loss Across Timepoints: [0.05328962 0.03976158]
[Validate] TP Loss Across Timepoints: [7.43238932 1.06244106]
[Validate] MAE of Mem Caps Across Timepoints: [0.52534905 0.47694714]
Epoch [2/20]
[Train] MAE Loss: 0.03140417902419965, TP Loss: 1.8092364107569059
[Validate] MAE Loss Across Timepoints: [0.05153271 0.0385108 ]
[Validate] TP Loss Across Timepoints: [7.2849645  0.95206788]
[Validate] MAE of Mem Caps Across Timepoints: [0.45077817 0.65922807]
Epoch [3/20]
[Train] MAE Loss: 0.03016704609617591, TP Loss: 1.7796531741817792
[Validate] MAE Loss Across Timepoints: [0.05114193 0.03807404]
[Validate] TP Loss Across Timepoints: [7.28689067 0.92457956]
[Validate] MAE of Mem Caps Across Timepoints: [0.4611794  0.65127847]
Epoch [4/20]
[Train] MAE Loss: 0.0295861534619083, TP Loss: 1.7682382089396318
[Validat

[Validate] MAE Loss Across Timepoints: [0.04732082 0.04342159]
[Validate] TP Loss Across Timepoints: [7.06113434 2.02854589]
[Validate] MAE of Mem Caps Across Timepoints: [0.45586485 0.40646846]
Epoch [10/20]
[Train] MAE Loss: 0.02594663224493464, TP Loss: 1.1866750836372375
[Validate] MAE Loss Across Timepoints: [0.04726782 0.04351747]
[Validate] TP Loss Across Timepoints: [6.96811574 1.990264  ]
[Validate] MAE of Mem Caps Across Timepoints: [0.42999338 0.38307034]
Epoch [11/20]
[Train] MAE Loss: 0.025910249321411054, TP Loss: 1.1851562534769375
[Validate] MAE Loss Across Timepoints: [0.04713336 0.04304344]
[Validate] TP Loss Across Timepoints: [6.98820292 1.93671964]
[Validate] MAE of Mem Caps Across Timepoints: [0.44407625 0.42377413]
Epoch [12/20]
[Train] MAE Loss: 0.025763408405085404, TP Loss: 1.1834316648542882
[Validate] MAE Loss Across Timepoints: [0.04702867 0.04288356]
[Validate] TP Loss Across Timepoints: [6.93155111 1.92898407]
[Validate] MAE of Mem Caps Across Timepoints:

[Train] MAE Loss: 0.026795811050881944, TP Loss: 1.3992107778787612
[Validate] MAE Loss Across Timepoints: [0.0385219  0.04186235]
[Validate] TP Loss Across Timepoints: [1.30207098 1.48606443]
[Validate] MAE of Mem Caps Across Timepoints: [0.51854808 0.60042809]
Epoch [19/20]
[Train] MAE Loss: 0.026761791637788215, TP Loss: 1.396154560893774
[Validate] MAE Loss Across Timepoints: [0.0381143  0.04132783]
[Validate] TP Loss Across Timepoints: [1.25296032 1.39512658]
[Validate] MAE of Mem Caps Across Timepoints: [0.54539847 0.59880398]
Epoch [20/20]
[Train] MAE Loss: 0.02664947845041752, TP Loss: 1.390572576969862
[Validate] MAE Loss Across Timepoints: [0.03823723 0.04137992]
[Validate] TP Loss Across Timepoints: [1.24824333 1.3822844 ]
[Validate] MAE of Mem Caps Across Timepoints: [0.62967833 0.64276896]

epochs finished with time:183.25348567962646

Current memory usage: 3316.21 MB

[Test] MAE Loss Across Timepoints: [0.03958857 0.04273708]
[Test] TP Loss Across Timepoints: [3.30753924 